# Crawling Web Berita CNN Indonesia

## Apa itu Crawling?

Crawling merupakan proses search engine untuk menemukan konten atau sesuatu situs halaman yang ada. Dalam bahasa kerennya crawling atau web crawling merupakan proses dimana search engine mengirimkan bot atau robot yang disebut (crawler atau spider) yang digunakan untuk menemukan konten-konten yang ada.

Yang dimaksud konten yaitu bervariasi, mulai dari halaman website yang saya lakukan ini, kemudian gambar, video, dokumen, dan lain sebagainya. Seperti halnya laba-laba, datang ke sebuah jaring dan melihat beberapa halaman website, kemudian mengikuti link yang terdapat di halaman website tersebut untuk mencari URL yang baru.

Ketika ada pengguna yang mencari sebuah konten di search engine dengan keyword tertentu, search engine akan mencarinya di indeks dan menentukan konten mana yang paling sesuai untuk pengguna tersebut.

## Proses Crawling

### Tool atau Library

In [2]:
# Import Library yang dibutuhkan
import requests as req
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from tqdm import tqdm

* **Request** digunakan untuk mengambil html/http dari sebuah website.
* **BeautifulSoup** berfungsi untuk mengambil data dari html/xml.
* **Time** berfungsi untuk memberikan jeda ketika ingin berpindah halaman.
* **Pandas** digunakan untuk membuat dataframe agar mudah dibaca.
* **tqdm** Untuk mentracking proses program

### Code Program

#### Fungsi Clean_text()

In [3]:
def clean_text(text):
	text = text.replace('\xa0', '',)
	return text.strip()

Fungsi untuk membersihkan text yang tidak diinginkan, atau mengganggu.

#### Fungsi scrape_news()

In [4]:
def scrape_news(soup):
	berita = {}
	texts = []
	# TODO:
	# ada struktur aneh https://www.cnnindonesia.com/olahraga/20240830134615-142-1139388/live-report-timnas-indonesia-vs-thailand-u-20
	
	berita["judul"] = soup.title.text

	if 'FOTO:' in berita["judul"]:
		div_content = soup.find("div", class_="detail-text text-cnn_black text-sm grow min-w-0")
		if div_content:
			full_text = div_content.get_text(strip=True)
			text = full_text.split('--', 1)[-1]
			text = text.split('var article')[0].strip()
			
			cleaned_text = clean_text(text)
			texts.append(cleaned_text)

	text_list = soup.find("div", class_="detail-text text-cnn_black text-sm grow min-w-0")
	for text in text_list.find_all("p"):
		if 'para_caption' not in text.get('class', []):
			cleaned_text = clean_text(text.text)
			texts.append(cleaned_text)
		
	berita["isi"] = "\n".join(texts)
	berita["tanggal"] = soup.find("div", class_="container !w-[1100px] overscroll-none").find_all("div")[1].find_all("div")[4].text
	berita["kategori"] = soup.find("a", attrs={"aria-label": "link description", "dtr-act": "kanal"}).text
	return berita

Pada fungsi ini berisikan proses pembedahan dan juga pengambilan data pada sebuah website. Mengambil data sesuai struktur HTML/web yang ingin diambil datanya.

#### Fungsi get_html()

In [5]:
def get_html(url):
	try:
		response = req.get(url).text
		return bs(response, "html5lib")
	
	except Exception as e:
		print(e)
		return ""

Fungsi get_html dengan parameter url digunakan untuk mengambil response atau isi html dari web. Untuk mengambil response tersebut dibutuhkan library request, dan juga BeautifulSoup untuk mendapatkan isi html.

#### Fungsi get_news()

In [6]:
def get_news(soup):
	container = soup.find("div", class_="container !w-[1100px] overscroll-none")
	news_list = container.find_all("article", class_="flex-grow")
	return news_list

Fungsi get_news berfungsi untuk mengambil semua berita yang ada pada web, yang kemudian didapat kumpulan url berita yang ada pada halaman web.

#### Main Crawling

In [7]:
def main(nasional: int, internasional: int, halaman: int=None):
	url = ["https://www.cnnindonesia.com/nasional/indeks/3/", "https://www.cnnindonesia.com/internasional/indeks/6/"]
	# url = ["https://www.cnnindonesia.com/indeks/2/"]
	news = []

	nasional_count = 0
	internasional_count = 0

	while url:
		url_now = url.pop()
		count = 1 # - n

		if halaman is None:
			halaman = int(get_html(url_now).find("div", class_="flex gap-5 my-8 items-center justify-center undefined").find_all("a")[-2].text)
			# print(f"Total page: {halaman}")

		for _ in range(halaman):
			page = f"{url_now}{count}"   
			soup = get_html(page)
			news_list = get_news(soup)
			
			for item in tqdm(news_list, desc=f"Processing page {count}"):
				# news_url = item.find('a')['href']
				news_url = "https://www.cnnindonesia.com/internasional/20240913095807-113-1144056/jepang-kerahkan-jet-tempur-saat-pesawat-rusia-mengitari-negeri-sakura"
				# print(news_url)
				result = scrape_news(get_html(news_url))

				if nasional_count >= nasional and internasional_count >= internasional:
					return news

				if result['kategori'] == 'Nasional' and nasional_count < nasional:
					news.append(result)
					nasional_count += 1
				elif result['kategori'] == 'Internasional' and internasional_count < internasional:
					news.append(result)
					internasional_count += 1
				
			count+=1
			time.sleep(1)
	
			if (nasional_count >= nasional and 'nasional' in url_now) or (internasional_count >= internasional and 'internasional' in url_now):
				break

	return news

Menyiapkan link/base url web berita yang ingin dicrawling, terdapat beberapa fungsi yang dipanggil yang sudah dibuat sebelumnya untuk mengambil informasi atau berita pada halaman website. Dalam code tersebut terdapat beberapa tahapan seperti fungsi:
* get_html
* get_news
* scrape_news

#### Main

In [8]:
news = main(nasional=0, internasional=1)

Processing page 1:  10%|█         | 1/10 [00:00<00:05,  1.57it/s]


Menjalankan program yang sudah dibuat dengan input berapa halaman yang ingin diambil.

### Convert Dataframe

In [9]:
df = pd.DataFrame.from_dict(news)
df
df.to_csv("data_100.csv", index=False)

In [10]:
print(df['isi'][0])

Jepang mengerahkan sejumlah jet tempur usai pesawat Rusia untuk pertama kalinya dalam lima tahun terbang mengitari Negeri Sakura.
Kementerian Pertahanan Jepang pada Jumat (13/9) menyatakan pesawat Tu-142 milik Rusia terbang dari laut yang memisahkan Jepang-Korea Selatan menuju wilayah Okinawa selatan sejak Kamis (12/9) pagi.

Pesawat-pesawat Kremlin itu kemudian menuju ke utara di atas Samudra Pasifik dan mengakhiri penerbangannya di Pulau Hokkaido utara.
Pesawat-pesawat ini tidak memasuki wilayah udara Jepang. Namun, mereka terbang di atas kawasan yang menjadi sengketa antara Jepang dan Rusia.
"Sebagai tanggapan, kami mengerahkan jet tempur Pasukan Bela Diri Udara (Angkatan Udara) dalam rangka keadaan darurat," demikian keterangan Kementerian Pertahanan Jepang, seperti dikutip AFP.

Ini merupakan kali pertama dalam lima tahun Rusia memutari Jepang dengan pesawatnya. Terakhir kali pesawat militer Kremlin mengitari Jepang yaitu pada 2019, kala bomber (pesawat pengebom) Rusia memasuki wi

In [11]:
# Library untuk data manipulation
import pandas as pd
from tqdm import tqdm
import re
import string

# Library untuk text preprocessing
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')

# Library untuk text vectorization/TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to C:\Users\LAB
[nltk_data]     SISTER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\LAB
[nltk_data]     SISTER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:
def clean_text(text):
	text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', ' ', text) # Menghapus https* and www*
	text = re.sub(r'@[^\s]+', ' ', text) # Menghapus username
	text = re.sub(r'[\s]+', ' ', text) # Menghapus tambahan spasi
	text = re.sub(r'#([^\s]+)', ' ', text) # Menghapus hashtags
	text = re.sub(r'rt', ' ', text) # Menghapus retweet
	text = text.translate(str.maketrans("","",string.punctuation)) # Menghapus tanda baca
	text = re.sub(r'\d', ' ', text) # Menghapus angka
	text = text.lower()
	text = text.encode('ascii','ignore').decode('utf-8') #Menghapus ASCII dan unicode
	text = re.sub(r'[^\x00-\x7f]',r'', text)
	text = text.replace('\n','') #Menghapus baris baru
	text = text.strip()
	return text
def stemming_indo(text):
	factory = StemmerFactory()
	stemmer = factory.create_stemmer()
	text = ' '.join(stemmer.stem(word) for word in text)
	return text
def clean_stopword(tokens):
	listStopword =  set(stopwords.words('indonesian'))
	removed = []
	for t in tokens:
		if t not in listStopword:
			removed.append(t)
	return removed

In [13]:
def preprocess_text(content):
	result = []
	for text in tqdm(content):
		cleaned_text = clean_text(text)
		tokens = nltk.tokenize.word_tokenize(cleaned_text)
		cleaned_stopword = clean_stopword(tokens)
		stemmed_text = stemming_indo(cleaned_stopword)
		result.append(stemmed_text)
	return result

df['cleaned_text'] = preprocess_text(df['isi'])

100%|██████████| 1/1 [00:00<00:00,  7.53it/s]


In [14]:
def tfidf_vsm(data, kategori):
	tfidf = TfidfVectorizer()
	tfidf_matrix = tfidf.fit_transform(data)
	feature_names = tfidf.get_feature_names_out()
	
	df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
	df_tfidf.insert(0, 'Kategori Berita', kategori.reset_index(drop=True))

	return tfidf, df_tfidf

# tfidf, df_tfidf = tfidf_vsm(df['cleaned_text'], df['kategori'])

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import pickle

In [18]:
with open('tugas3/lr_model.pkl', 'rb') as file:  
    # Call load method to deserialze 
    model = pickle.load(file)
with open('tugas2/tfidf_vsm.pkl', 'rb') as file:  
    # Call load method to deserialze 
    tfidf = pickle.load(file)

tfidf_matrix = tfidf.transform(df['cleaned_text'])

In [20]:
# Retrain the logistic regression model with the updated feature matrix
model.predict(tfidf_matrix)


c:\Users\LAB SISTER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0])